# LangChain Tools: Knowledge and Capability Augmentation

- Author: Martin Fockedey

## Introduction

Large Language Models (LLMs) have remarkable capabilities, but they also have fundamental limitations:
- **Knowledge cutoff**: Training data ends at a specific date
- **No real-time information**: Cannot access current events, live data, or dynamic content
- **Mathematical limitations**: Struggle with precise calculations
- **No external interactions**: Cannot query databases, call APIs, or browse the web

**LangChain Tools** solve these limitations by allowing LLMs to:
1. **Augment knowledge**: Access external information sources (web search, databases, APIs)
2. **Augment capabilities**: Perform precise operations (calculations, data processing, custom logic)

In this notebook, we'll explore both types of augmentation with practical examples using Mistral AI.

## Environment Setup

Set up the environment.

[Note]
- You'll need a Mistral AI API key (see your Teams group channel).
- For web search examples, you'll also need a Tavily API key (free tier available at https://tavily.com)
- Store your API keys in a `.env` file as `MISTRAL_API_KEY` and `TAVILY_API_KEY`.

In [6]:
# Install required packages 
!pip install mistralai python-dotenv requests

  Obtaining dependency information for mistralai from https://files.pythonhosted.org/packages/fe/76/4ce12563aea5a76016f8643eff30ab731e6656c845e9e4d090ef10c7b925/mistralai-1.9.11-py3-none-any.whl.metadata
  Obtaining dependency information for eval-type-backport>=0.2.0 from https://files.pythonhosted.org/packages/19/d8/2a1c638d9e0aa7e269269a1a1bf423ddd94267f1a01bbe3ad03432b67dd4/eval_type_backport-0.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for invoke<3.0.0,>=2.2.0 from https://files.pythonhosted.org/packages/32/4b/b99e37f88336009971405cbb7630610322ed6fbfa31e1d7ab3fbf3049a2d/invoke-2.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for pyyaml<7.0.0,>=6.0.2 from https://files.pythonhosted.org/packages/da/e3/ea007450a105ae919a72393cb06f122f288ef60bba2dc64b26e2646fa315/pyyaml-6.0.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/442.8 kB ? eta -:--:--
    --------------------------------------- 10.2/442.8 kB ? eta -:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires PyYAML==6.0.1, but you have pyyaml 6.0.3 which is incompatible.
datasets 2.12.0 requires huggingface-hub<1.0.0,>=0.11.0, but you have huggingface-hub 1.0.1 which is incompatible.
transformers 4.32.1 requires huggingface-hub<1.0,>=0.15.1, but you have huggingface-hub 1.0.1 which is incompatible.
transformers 4.32.1 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.22.1 which is incompatible.


In [7]:
# Environment setup using official mistralai client (no LangChain)
import os, json, inspect
from dotenv import load_dotenv
from typing import Callable, List

from mistralai import Mistral, UserMessage, ToolMessage

load_dotenv(override=True)

API_KEY = os.getenv("MISTRAL_API_KEY")
if not API_KEY:
    print("Warning: MISTRAL_API_KEY is not set.")

MODEL_NAME = os.getenv("MISTRAL_MODEL", "mistral-small")
TEMPERATURE = float(os.getenv("MISTRAL_TEMPERATURE", "0.0"))
client = Mistral(api_key=API_KEY)

print(f"Environment loaded! Using model: {MODEL_NAME}")

def build_tool_spec(func: Callable):
    """Build a tool spec dict from a plain python function.
    Assumes all parameters are strings unless type annotation gives something else.
    """
    sig = inspect.signature(func)
    props = {}
    required = []
    for name, param in sig.parameters.items():
        ann = param.annotation
        ann_type = "string"
        if ann in (int, float):
            ann_type = "number"
        props[name] = {"type": ann_type}
        if param.default is inspect._empty:
            required.append(name)
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip()[:800],
            "parameters": {
                "type": "object",
                "properties": props,
                "required": required
            }
        }
    }

def run_tool_chat(user_content: str, funcs: List[Callable], model: str = MODEL_NAME, temperature: float = TEMPERATURE):
    """Send a user message, handle any tool calls, return final answer string."""
    messages = [UserMessage(role="user", content=user_content)]
    tool_specs = [build_tool_spec(f) for f in funcs]
    first = client.chat.complete(model=model, messages=messages, tools=tool_specs, temperature=temperature)
    msg = first.choices[0].message
    tool_calls = msg.tool_calls or []
    if not tool_calls:
        return msg.content
    messages.append(msg)
    for tc in tool_calls:
        # Parse args and execute matching function
        args = json.loads(tc.function.arguments)
        fn = next((f for f in funcs if f.__name__ == tc.function.name), None)
        if fn is None:
            result = f"Error: function {tc.function.name} not implemented"
        else:
            try:
                result = fn(**args)
            except Exception as e:
                result = f"Error executing {tc.function.name}: {e}".strip()
        print(f"Tool {tc.function.name}({args}) -> {str(result)[:160]}")
        messages.append(ToolMessage(role="tool", content=str(result), name=tc.function.name, tool_call_id=tc.id))
    final = client.chat.complete(model=model, messages=messages, temperature=temperature)
    return final.choices[0].message.content

Environment loaded! Using model: mistral-small


## Part 1: Capability Augmentation

Capability augmentation gives LLMs the ability to perform operations they cannot do natively, such as precise calculations or string manipulation.

### Example 1.1: Calculator Tool

LLMs are notoriously bad at arithmetic. Let's create a calculator tool to fix this.

In [8]:
# First, let's see the LLM fail at math
print("=== Without Calculator Tool ===")
response = client.chat.complete(model=MODEL_NAME, messages=[UserMessage(role="user", content="What is 1247256 * 548936 / 452661?")], temperature=TEMPERATURE)
print(response.choices[0].message.content)
print()

=== Without Calculator Tool ===


### Understanding the Problem

First, I need to understand what's being asked. The problem is:

**What is 1247256 × 548936 / 452661?**

This is a combination of multiplication and division. To break it down:
1. Multiply two large numbers: 1,247,256 and 548,936.
2. Divide the result from step 1 by another large number: 452,661.

### Step 1: Breaking Down the Multiplication

Multiplying two large numbers like 1,247,256 and 548,936 seems daunting. I recall the **distributive property of multiplication over addition**, which allows breaking down the problem into simpler parts.

#### Using the FOIL Method (or Binomial Expansion)

I think of the numbers in terms of their components:
- **First Number (A)**: 1,247,256
- **Second Number (B)**: 548,936

I can express B as (500,000 + 40,000 + 8,000 + 900 + 30 + 6).

Now, I can multiply A by each component of B and then add the results.

1. **Multiply A by 500,000**:
   \[ 1,247,256 × 500,000 = 1,247,256 × 5 × 10^5 = 6,236,280 × 10^5 = 623,628,000

In [9]:
# Calculator tool (plain function)
def calculator(expression: str) -> str:
    """Evaluate a mathematical expression safely for arithmetic purposes.
    Args:
        expression: A math expression like "1247 * 8936 + 4521".
    Returns:
        Result as a string or an error message.
    """
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}".strip()

# Quick direct test
print("Testing calculator tool directly:")
print(calculator("1247256 * 548936 / 452661"))

Testing calculator tool directly:
1512530.8334846606


In [10]:
# Calculator via mistralai tool calling
print("=== Calculator via official tool calling ===")
answer = run_tool_chat("What is 1247256 * 548936 / 452661?", [calculator])
print("Final Answer:", answer)

=== Calculator via official tool calling ===


Tool calculator({'expression': '1247256 * 548936 / 452661'}) -> 1512530.8334846606


Final Answer: The result of \( \frac{1247256 \times 548936}{452661} \) is approximately **1,512,530.8335**.


### Example 1.2: Vowel Counter Tool

Remember the vowel counting exercise from Notebook 03? LLMs are terrible at it. Let's create a tool to count vowels accurately.

In [28]:
# Vowel counter tool (plain function)
def count_vowels(text: str) -> str:
    """Count vowels (a,e,i,o,u,y) in text, case-insensitive.
    Args:
        text: Input string.
    Returns:
        Summary string with total and per-vowel breakdown.
    """
    vowels = "aeiouy"
    lower = text.lower()
    counts = {v: lower.count(v) for v in vowels}
    total = sum(counts.values())
    breakdown = ", ".join([f"{v}: {c}" for v, c in counts.items() if c > 0])
    return f"Total vowels: {total} ({breakdown})"

# Direct tests
for word in ["intelligence", "artificielle", "programmation"]:
    print(word, "=>", count_vowels(word))

intelligence => Total vowels: 5 (e: 3, i: 2)
artificielle => Total vowels: 6 (a: 1, e: 2, i: 3)
programmation => Total vowels: 5 (a: 2, i: 1, o: 2)


In [29]:
# Multi-tool capability example using mistralai tool calling
print("=== Multi-Tool (calculator + count_vowels) ===")
capability_answer = run_tool_chat(
    "How many vowels are in the word 'Constantinople'? Then multiply that number by 15.",
    [calculator, count_vowels]
)
print("Final Answer:", capability_answer)

=== Multi-Tool (calculator + count_vowels) ===


Tool count_vowels({'text': 'Constantinople'}) -> Total vowels: 5 (a: 1, e: 1, i: 1, o: 2)


Final Answer: The word "Constantinople" contains 5 vowels. Multiplying that number by 15 gives:

5 * 15 = 75


## Part 2: Knowledge Augmentation

Knowledge augmentation allows LLMs to access information beyond their training data: real-time web content, databases, APIs, and more.

### Example 2.1: Information search with Wikipedia


In [47]:
import wikipedia

def search_wikipedia(query: str) -> str:
    """Search Wikipedia for information about a topic.
    Args:
        query: The search query or topic to look up on Wikipedia.
    Returns:
        Summary of the Wikipedia article or error message.
    """
    try:
        # Set language to English
        wikipedia.set_lang("en")
        # Get summary (first few sentences)
        summary = wikipedia.summary(query, sentences=3)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        # Multiple possible pages - return the options
        options = ", ".join(e.options[:5])
        return f"Multiple results found. Please be more specific. Options: {options}"
    except wikipedia.exceptions.PageError:
        return f"No Wikipedia page found for '{query}'"
    except Exception as e:
        return f"Wikipedia search error: {e}"

# Test the tool
print("=== Testing Wikipedia Search Tool ===")
print(search_wikipedia("Artificial Intelligence"))
print()
print(search_wikipedia("Python programming"))

=== Testing Wikipedia Search Tool ===
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go).

Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associat

In [49]:
# Wikipedia search via mistralai tool calling
print("=== Wikipedia Search Tool Call ===")
wiki_answer = run_tool_chat(
    "Does Brussels have a governement?",
    [search_wikipedia]
)
print("Final Answer:", wiki_answer)

=== Wikipedia Search Tool Call ===


Final Answer: Yes, Brussels has a government. The Brussels-Capital Region is one of the three regions of Belgium, alongside Flanders and Wallonia. The government of the Brussels-Capital Region is responsible for managing the region's affairs, including urban development, public works, transportation, and environmental policies.

The Brussels-Capital Region has its own parliament and executive bodies, which are distinct from those of the federal government of Belgium and the governments of the other regions. The government of the Brussels-Capital Region is led by a Minister-President and a team of ministers who are responsible for various policy areas.

Brussels is also the de facto capital of the European Union, and as such, it hosts many of the EU's institutions, including the European Commission, the Council of the European Union, and the European Council. These institutions have their own governance structures and are separate from the government of the Brussels-Capital Region.


### Example 2.2: SQL Database Query Tool

Let's create a tool that queries a simple SQLite database. This demonstrates how LLMs can interact with structured data sources.

In [36]:
# Create a simple SQLite database with student data
import sqlite3

# Create in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create table and insert sample data
cursor.execute('''
CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    grade REAL,
    subject TEXT
)
''')

students_data = [
    (1, 'Alice', 20, 85.5, 'Computer Science'),
    (2, 'Bob', 21, 92.0, 'Mathematics'),
    (3, 'Charlie', 19, 78.5, 'Physics'),
    (4, 'Diana', 22, 88.0, 'Computer Science'),
    (5, 'Eve', 20, 95.5, 'Mathematics'),
]

cursor.executemany('INSERT INTO students VALUES (?,?,?,?,?)', students_data)
conn.commit()

print("Sample database created with 5 students")

Sample database created with 5 students


In [42]:
# Database query tool (plain function)
def query_students_db(query: str) -> str:
    """Execute a read-only SELECT query against the Computer Science students in-memory 
    students grade table that was created with 
    CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    age INTEGER,
    grade REAL,
    subject TEXT).
    Args:
        query: SQL SELECT statement.
    Returns:
        Formatted rows or error message.
    """
    try:
        if not query.strip().upper().startswith("SELECT"):
            return "Only SELECT queries allowed"
        cursor.execute(query)
        rows = cursor.fetchall()
        if not rows:
            return "No results"
        return "\n".join(str(r) for r in rows)
    except Exception as e:
        return f"DB error: {e}".strip()

# Quick test
print(query_students_db("SELECT name, grade FROM students WHERE grade > 85"))

('Alice', 85.5)
('Bob', 92.0)
('Diana', 88.0)
('Eve', 95.5)


In [43]:
# Database + calculator via mistralai
print("=== Database Tool Call ===")
db_answer = run_tool_chat(
    "What is the average grade of all Computer Science students? Show the numeric average only.",
    [query_students_db, calculator]
)
print("Final Answer:", db_answer)

=== Database Tool Call ===


Tool query_students_db({'query': "SELECT AVG(grade) FROM students WHERE subject = 'Computer Science';"}) -> (86.75,)


Final Answer: The average grade of all Computer Science students is **86.75**.


### Example 2.3: Custom API Tool

Let's create a tool that fetches data from a public API. We'll use a simple weather-like API concept.

In [44]:
import requests, json

# City info tool (plain function)
def get_city_info(city: str) -> str:
    """Lookup basic city info (lat, lon, population) via Open-Meteo geocoding API.
    Args:
        city: City name.
    Returns:
        Multi-line string with details or error.
    """
    try:
        url = f"https://geocoding-api.open-meteo.com/v1/search?name={city}&count=1&language=en&format=json"
        r = requests.get(url, timeout=8)
        r.raise_for_status()
        data = r.json()
        results = data.get("results", [])
        if not results:
            return f"City '{city}' not found"
        res = results[0]
        return (
            f"City: {res.get('name')}\n" \
            f"Country: {res.get('country')}\n" \
            f"Latitude: {res.get('latitude')}\n" \
            f"Longitude: {res.get('longitude')}\n" \
            f"Population: {res.get('population', 'N/A')}"
        )
    except Exception as e:
        return f"City info error: {e}".strip()

# Quick test
print(get_city_info("Brussels"))

City: Brussels
Country: Belgium
Latitude: 50.85045
Longitude: 4.34878
Population: 1019022


In [45]:
# City info + calculator via mistralai
print("=== City Info Tool Call ===")
city_answer = run_tool_chat(
    "What is the population of Paris? Multiply it by 2 and show the result.",
    [get_city_info, calculator]
)
print("Final Answer:", city_answer)

=== City Info Tool Call ===


Tool get_city_info({'city': 'Paris'}) -> City: Paris
Country: France
Latitude: 48.85341
Longitude: 2.3488
Population: 2138551
Tool calculator({'expression': 'population * 2'}) -> Error: name 'population' is not defined


Final Answer: It appears there was an error in the calculation. Let me try again.

The population of Paris is 2,138,551. Multiplying this by 2 gives:

2,138,551 * 2 = 4,277,102


## Part 3: Combining Multiple Tools

The real power of tools comes from combining them. Let's create an agent with access to all our tools and see it solve complex, multi-step problems.

In [46]:
# Comprehensive multi-tool example via mistralai tool calling
print("=== Comprehensive Multi-Tool Call ===")
complex_final = run_tool_chat(
    """Find the student with the highest grade, count vowels in their name, multiply that count by the grade, then compare that product to twice the population of Brussels (if population known).""",
    [query_students_db, count_vowels, calculator, get_city_info]
)
print("Final Answer:", complex_final)

=== Comprehensive Multi-Tool Call ===


Tool query_students_db({'query': 'SELECT name, grade FROM students ORDER BY grade DESC LIMIT 1;'}) -> ('Eve', 95.5)
Tool get_city_info({'city': 'Brussels'}) -> City: Brussels
Country: Belgium
Latitude: 50.85045
Longitude: 4.34878
Population: 1019022
Tool get_city_info({'city': 'Brussels'}) -> City: Brussels
Country: Belgium
Latitude: 50.85045
Longitude: 4.34878
Population: 1019022


Final Answer: ### Analysis

1. **Student with the highest grade**: Eve (Grade: 95.5)
2. **Count of vowels in the student's name**:
   - Name: "Eve"
   - Vowels: E, e (2 vowels)
3. **Multiply the vowel count by the grade**:
   - Calculation: 2 * 95.5 = 191
4. **Twice the population of Brussels**:
   - Population of Brussels: 1,019,022
   - Calculation: 2 * 1,019,022 = 2,038,044

### Comparison
The product of the vowel count and the grade (191) is significantly smaller than twice the population of Brussels (2,038,044).


## Exercise: Create Your Own Tool

Design and implement a custom tool for one of these scenarios:

### Option 1: Date/Time Tool
Create a tool that answers questions about dates and times:
- Days between two dates
- Day of the week for any date
- Add/subtract days from a date
- Time zone conversions

### Option 2: Text Analysis Tool
Create a tool that analyzes text:
- Word count and unique word count
- Reading time estimation
- Sentence count
- Most common words

### Option 3: Unit Converter Tool
Create a tool that converts between units:
- Temperature (Celsius, Fahrenheit, Kelvin)
- Length (meters, feet, miles, km)
- Weight (kg, lbs, oz)
- Currency (mock exchange rates)
